In [1]:
import os
import numpy as np
import multiprocessing as mp
from mg5_tools import semilla, run_mg5_file


UFO_name = "Mod2_VLQ_UFO"
cases = ["woRHC"]

mass_step = 500 # GeV
g_u_step = 1/4

# Leptoquark parameters
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.arange(0.75, 3.25 + g_u_step , g_u_step) 

# Leptoquark parameters (fixed)
M_U = np.array([1000,2000])
g_U = np.array([1.75])

# Z' parameters
mass_constraints_zp = {"zp_upper_limit": M_U*np.sqrt(3./2.), "zp_lower_limit": M_U*np.sqrt(1./2.)}
g_zp_step = 1/4
g_zp = np.arange(0, 3.5 + g_zp_step, g_zp_step)

output_folder= os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "SIMULATIONS", f"lq_zp")
os.makedirs(output_folder, exist_ok=True)

n_events = 0.8e6

nevts = 50e3
n_runs = int(n_events/nevts)
n_cores = mp.cpu_count()

n_workers = 3 

n_cores = int(n_cores/n_workers)

if n_cores == 1:
    raise ValueError("n_cores must be greater than 1")




In [2]:
headers = {
"non-res_tau_tau" :
f"""
import model {UFO_name}
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0 QCD = 0 $$ zp
""",
"zp_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > zp > ta ta @0 QED = 0 QCD = 0 
""",
"total_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0 QCD = 0 
"""
}

kin_gen_cuts = {
    "cut_decays" : True,
    "ptb" : 30,
    "ptj" : 20,
    "ptl" : 20,
    "etab" : 2.5,
    "pt_min_pdg" : "{15: 30}",
    "eta_max_pdg" : '{15: 2.5}',
    "mmll" : 20
}

In [3]:
def prepare_proc_script(x, output_folder):
    case = x[0] 
    header = x[1]
    mass_constraint_zp = x[2][0]
    gz = x[3]
    gu = x[4]
    m = x[5]
    label = f"mu_{m}_gu{gu}_gzp_{gz}"
    mg5_output_folder = os.path.join(output_folder, case, mass_constraint_zp, header, label.replace(".", "_"))
    os.makedirs(mg5_output_folder, exist_ok=True)

    proc_file_path = os.path.join(mg5_output_folder, f"proc_script.mg5")
    with open(proc_file_path, 'w') as f:
        f.write(headers[header])
        f.write(f"output {mg5_output_folder} -nojpeg\n")

    # check if the process has already been generated
    if not os.path.exists(proc_file_path):
        raise Exception(f"Process {proc_file_path} not generated")
    return run_mg5_file(proc_file_path)




In [4]:
from itertools import product

prod = list(product(cases, headers.keys(), mass_constraints_zp.items(), g_zp, g_U, M_U))
all(list( map ( prepare_proc_script, prod, [output_folder]*len(prod) ) ))


Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC
Madg

False

In [5]:
seeds =[]

def generate_events(x, output_folder):
    case = x[0] 
    header = x[1]
    mass_constraint_zp = x[2][0]
    gz = x[3]
    gu = x[4]
    m = x[5]

    seed = semilla(seeds)
    label = f"mu_{m}_gu{gu}_gzp_{gz}"
    param_card = os.path.join("Param_Cards", case, mass_constraint_zp, f"param_card_{label}.dat")

    # check if param card exists
    if not os.path.exists(param_card):
        raise Exception(f"Param card {param_card} not generated")
    
    delphes_card = os.path.join(os.getcwd(), "delphes_card_CMS-0.7bTag.dat")
    pythia_card = os.path.join(os.getcwd(), "pythia8_card.dat")

    mg5_output_folder = os.path.join(output_folder, case, mass_constraint_zp , header, label.replace(".", "_"))

    # check if the process has already been generated
    if not os.path.exists(mg5_output_folder):
        raise Exception(f"Process {mg5_output_folder} not generated")
    
    f = open(os.path.join(mg5_output_folder, f"run_{seed}.mg5"), 'w')
    f.write(f"launch {mg5_output_folder} -m\n")
    f.write(f"{n_cores}\n") 
    f.write("shower = Pythia8 \n")
    f.write("detector = Delphes \n")
    f.write("done \n")
    f.write(f"set nevents {nevts}\n")
    f.write(f"set iseed {seed}\n")
    [f.write(f"set {key} {value}\n") for key, value in kin_gen_cuts.items()]
    f.write(f"{param_card}\n")
    f.write(f"{delphes_card}\n")
    f.write(f"{pythia_card}\n")
    f.write("done \n")
    f.close()

    return run_mg5_file(os.path.join(mg5_output_folder, f"run_{seed}.mg5"))
    


In [6]:
import multiprocessing as mp

prod = list(product(cases, headers.keys(), mass_constraints_zp.items(), g_zp, g_U, M_U))

for run in range(n_runs):
    with mp.Pool(n_workers) as pool:
        list( pool.map ( generate_events, prod, [output_folder]*len(prod) ) )
    

Madgraph is not installed in ../../../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC


Exception: Param card Param_Cards/woRHC/zp_upper_limit/param_card_mu_2000_gu0.75_gzp_0.0.dat not generated

In [ ]:
raise Exception("STOP HERE")

Exception: STOP HERE

In [ ]:
from itertools import product

log_file = open (os.path.join(os.getcwd(), "generation.log"), 'w')

for m, g in product(M_U, g_U):
    seeds=[]
    for header in headers:
        if "lq_lq" == header: continue
        if "lq_ta" == header: continue
        label = f'M{m}_gU{g:.4f}'.replace('.', '_')
        mg5_outpuut_folder = os.path.join(output_folder, header,label)
        os.makedirs(mg5_outpuut_folder, exist_ok=True)
        
        proc_file_path = os.path.join(mg5_outpuut_folder, f"proc_script.mg5")
        with open(proc_file_path, 'w') as f:
            f.write(headers[header])
            f.write(f"output {mg5_outpuut_folder} -nojpeg\n")
        run_mg5_file(proc_file_path)

        param_card = os.path.join(param_cards_folder, f"param_card_{label}.dat")
        delphes_card = os.path.join(os.getcwd(), "delphes_card_CMS-0.7bTag.dat")
        pythia_card = os.path.join(os.getcwd(), "pythia8_card.dat")
        
        if not os.path.isfile(param_card):
            raise Exception(f"Param card {os.path.basename(param_card)} not found")
        
        for i in range(n_runs):
            
            f = open(os.path.join(mg5_outpuut_folder, f"run_{i}.mg5"), 'w')
            f.write(f"launch {mg5_outpuut_folder} -m\n")
            f.write(f"{n_cores}\n") 
            f.write("shower = Pythia8 \n")
            f.write("detector = Delphes \n")
            f.write("done \n")
            f.write(f"set nevents {nevts}\n")
            
            seed = semilla(seeds)
            f.write(f"set iseed {seed}\n")
            [f.write(f"set {key} {value}\n") for key, value in kin_gen_cuts.items()]
            f.write(f"{param_card}\n")
            f.write(f"{delphes_card}\n")
            f.write(f"{pythia_card}\n")
            f.write("done \n")
            f.close()

            run_mg5_file(os.path.join(mg5_outpuut_folder, f"run_{i}.mg5"))
    log_file.write(f"Done for M_U = {m}, g_U = {g}")
log_file.close()